In [1]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path

print(Path().cwd())
os.chdir(Path().cwd().parent)
print(Path().cwd())

c:\0_Main\2_Quantitative_ultrasound\2_github\BSC\notebook
c:\0_Main\2_Quantitative_ultrasound\2_github\BSC


### Results

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

def read_bsc_results(sample_name: str, results_folder: str = "data/results") -> dict:
    """
    Read BSC results from CSV files and convert them to pandas DataFrames.
    
    Args:
        sample_name (str): Name of the sample (e.g., 'UKDCEUS030')
        results_folder (str): Path to the results folder
        
    Returns:
        dict: Dictionary containing:
            - 'depths': DataFrame with depth values in cm
            - 'frequencies': DataFrame with frequency values in MHz
            - 'power_ratio': DataFrame with power ratio values in dB
    """
    # Construct paths
    base_path = Path(results_folder) / sample_name
    depths_path = base_path / "depths_cm.csv"
    freqs_path = base_path / "frequencies_MHz.csv"
    power_path = base_path / "power_ratio.csv"
    
    # Read depths (single column)
    depths_df = pd.read_csv(depths_path, header=None, names=['Depth (cm)'])
    
    # Read frequencies (single column)
    freqs_df = pd.read_csv(freqs_path, header=None, names=['Frequency (MHz)'])
    
    # Read power ratio (multiple columns)
    # Each row represents a depth, each column represents a frequency
    power_df = pd.read_csv(power_path, header=None)
    
    # Set index names
    depths_df.index.name = 'Index'
    freqs_df.index.name = 'Index'
    power_df.index.name = 'Depth Index'
    
    # Create proper column names for power ratio DataFrame
    # Using the frequencies as column names
    power_df.columns = [f'{freq:.3f} MHz' for freq in freqs_df['Frequency (MHz)']]
    
    # Create a MultiIndex DataFrame for power ratio
    # This makes it easier to understand which depth and frequency each value corresponds to
    power_df_multi = pd.DataFrame(
        power_df.values,
        index=pd.Index(depths_df['Depth (cm)'], name='Depth (cm)'),
        columns=pd.Index(freqs_df['Frequency (MHz)'], name='Frequency (MHz)')
    )
    
    return {
        'depths': depths_df,
        'frequencies': freqs_df,
        'power_ratio': power_df,
        'power_ratio_multi': power_df_multi
    }

# Example usage:
sample_name = "UKDCEUS030"
results = read_bsc_results(sample_name)

# Access the DataFrames
depths_df = results['depths']
frequencies_df = results['frequencies']
power_ratio_df = results['power_ratio']
power_ratio_multi_df = results['power_ratio_multi']

# Print basic information
print("\nDepths DataFrame:")
print(depths_df.head())
print("\nDepths shape:", depths_df.shape)

print("\nFrequencies DataFrame:")
print(frequencies_df.head())
print("\nFrequencies shape:", frequencies_df.shape)

print("\nPower Ratio DataFrame (standard format):")
print(power_ratio_df.head())
print("\nPower Ratio shape:", power_ratio_df.shape)

print("\nPower Ratio DataFrame (multi-index format):")
print(power_ratio_multi_df.head())
print("\nPower Ratio Multi shape:", power_ratio_multi_df.shape)

# Basic statistics
print("\nDepth range (cm):", depths_df['Depth (cm)'].min(), "to", depths_df['Depth (cm)'].max())
print("Frequency range (MHz):", frequencies_df['Frequency (MHz)'].min(), "to", frequencies_df['Frequency (MHz)'].max())
print("Power ratio range (dB):", power_ratio_df.values.min(), "to", power_ratio_df.values.max())


Depths DataFrame:
       Depth (cm)
Index            
0           3.598
1           3.665
2           3.732
3           3.799
4           3.866

Depths shape: (57, 1)

Frequencies DataFrame:
       Frequency (MHz)
Index                 
0                0.000
1                0.234
2                0.469
3                0.703
4                0.938

Frequencies shape: (33, 1)

Power Ratio DataFrame (standard format):
             0.000 MHz  0.234 MHz  0.469 MHz  0.703 MHz  0.938 MHz  1.172 MHz  \
Depth Index                                                                     
0            20.519263  13.870617   8.329126   9.364098  12.242630  16.847362   
1            19.661406  12.218734   6.224437   8.660531  14.299297  22.069711   
2            17.653337   7.619526  -5.431204  -1.570075   7.799980  19.082509   
3            17.804626   7.878391  -5.291058  -1.753478   9.285759  20.845387   
4            16.806600   5.867654 -13.040341  -7.740984   4.151685  16.397522   

         